In [ ]:
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union

# Set precision
getcontext().prec = 30
scale_factor = Decimal("1")

# Base coordinates
trunk_w = 0.15
trunk_h = 0.2
base_w = 0.7
mid_w  = 0.4
top_w  = 0.25
tip_y = 0.8
tier_1_y = 0.5
tier_2_y = 0.25
base_y = 0.0
trunk_bottom_y = -trunk_h

BASE_COORDS = np.array([
    (0.0, tip_y),
    (top_w / 2, tier_1_y),
    (top_w / 4, tier_1_y),
    (mid_w / 2, tier_2_y),
    (mid_w / 4, tier_2_y),
    (base_w / 2, base_y),
    (trunk_w / 2, base_y),
    (trunk_w / 2, trunk_bottom_y),
    (-(trunk_w / 2), trunk_bottom_y),
    (-(trunk_w / 2), base_y),
    (-(base_w / 2), base_y),
    (-(mid_w / 4), tier_2_y),
    (-(mid_w / 2), tier_2_y),
    (-(top_w / 4), tier_1_y),
    (-(top_w / 2), tier_1_y),
])

class ChristmasTree:
    def __init__(self, center_x="0", center_y="0", angle="0"):
        self.center_x = float(center_x)
        self.center_y = float(center_y)
        self.angle = float(angle)
        self._polygon = None

    @property
    def polygon(self):
        if self._polygon is None:
            rad = np.radians(self.angle)
            c, s = np.cos(rad), np.sin(rad)
            R = np.array([[c, -s], [s, c]])
            new_coords = BASE_COORDS @ R.T + np.array([self.center_x, self.center_y])
            self._polygon = Polygon(new_coords)
        return self._polygon

def parse_csv(csv_path):
    df = pd.read_csv(csv_path)
    df["x"] = df["x"].astype(str).str.strip().str.lstrip("s")
    df["y"] = df["y"].astype(str).str.strip().str.lstrip("s")
    df["deg"] = df["deg"].astype(str).str.strip().str.lstrip("s")
    df[["group_id", "item_id"]] = df["id"].str.split("_", n=2, expand=True)
    
    dict_of_tree_list = {}
    for group_id, group_data in df.groupby("group_id"):
        tree_list = [
            ChristmasTree(center_x=row["x"], center_y=row["y"], angle=row["deg"])
            for _, row in group_data.iterrows()
        ]
        dict_of_tree_list[int(group_id)] = tree_list
    return dict_of_tree_list

def get_side(trees):
    min_x, min_y = float('inf'), float('inf')
    max_x, max_y = float('-inf'), float('-inf')
    for t in trees:
        bounds = t.polygon.bounds
        min_x = min(min_x, bounds[0])
        min_y = min(min_y, bounds[1])
        max_x = max(max_x, bounds[2])
        max_y = max(max_y, bounds[3])
    return max(max_x - min_x, max_y - min_y)

base_dict = parse_csv("submission.csv")
sa_dict = parse_csv("submission_sa_parallel.csv")

print(f"{'N':<5} {'Base Side':<15} {'SA Side':<15} {'Diff':<15}")
print("-" * 55)

improved_count = 0
for n in sorted(base_dict.keys()):
    if n > 20: break # Only checked up to 20
    
    side_base = get_side(base_dict[n])
    side_sa = get_side(sa_dict[n])
    diff = side_base - side_sa
    
    if diff > 1e-9:
        print(f"{n:<5} {side_base:<15.6f} {side_sa:<15.6f} {diff:<15.6e} *")
        improved_count += 1
    else:
        print(f"{n:<5} {side_base:<15.6f} {side_sa:<15.6f} {diff:<15.6e}")

print(f"\nTotal improved groups: {improved_count}")
